# Methodology
* GenerateDummyDataSet 
* Define Models [ Multiple Models  ]
* Define Loss , Optimizer 
* Training Loop
* Visualization Plot Loss , Model , Accuracy 


# GENERATE DATASET

In [4]:
import numpy as np
import torch
from classificationUtils import generate_dataset, plot_dataset

# generate dataset for clasification
X1,Y1,X2,Y2,X,Y = generate_dataset(n=100,r12=2,r34=5)
print(X.shape, Y.shape) #torch.Size([200, 2]) torch.Size([200, 1])

plot_dataset(X[:,0],X[:,1],Y[:,0])


torch.Size([200, 2]) torch.Size([200, 1])
 X_np (200,), Y_np (200,), Z_np (200,)


# DEFINE MODELS

In [7]:
from torch import nn
import torch
torch.manual_seed(42) # Very Important for reproducibility
from plotly.subplots import make_subplots
from classificationUtils import plotAllModels
import numpy as np 

torch.manual_seed(42)
modelConfigurations = [
    {
        "desc" : "2L_16hidden 2 Sigmoid ",
        "model": nn.Sequential(nn.Linear(2,16),nn.Sigmoid(), nn.Linear(16,1),nn.Sigmoid()),
        "static": nn.Sequential(nn.Linear(2,16),nn.Sigmoid(), nn.Linear(16,1),nn.Sigmoid())
    },
    {
        "desc" : "1 Layer + Sigmoid ",
        "model": nn.Sequential(nn.Linear(2,1),nn.Sigmoid()),
        "static":  nn.Sequential(nn.Linear(2,1),nn.Sigmoid()),
    },
    {
        "desc" : "2L_16hidden 1 Sigmoid ",
        "model": nn.Sequential(nn.Linear(2,16), nn.Linear(16,1),nn.Sigmoid()),
        "static": nn.Sequential(nn.Linear(2,16), nn.Linear(16,1),nn.Sigmoid()),
    },
      
]
# Synchronize the weights between model and static , keeping static copy for comparision
for config in modelConfigurations:
    config['static'].load_state_dict(config['model'].state_dict())

plotAllModels(modelConfigurations,ncols=3)


torch.Size([10000, 1]) torch.Size([10000, 1]) torch.Size([10000, 2])
 rows 1 cols 3
1 1
1 2
1 3


#  Loss + Optimizer

In [8]:
from time import sleep
from IPython import display
import pandas as pd
import matplotlib.pyplot as plt

n_models = 1 #modelConfigurations.__len__() 

def datapoint_dataframe():
    X_np = X[:,0].numpy()
    Y_np = X[:,1].numpy()
    Z_np = Y[:,0].numpy()
    data = {'X': X_np, 'Y': Y_np, 'Z': Z_np}
    df = pd.DataFrame(data)
    return df

df = datapoint_dataframe()

losses = []
accuracies=[]
epochs = 500

for i in range(n_models):
    model = modelConfigurations[i]['model']
    print(f"Satarting Model {i} {model}")
    
    criteria =  nn.BCELoss()
    optimizer = torch.optim.Adam(model.parameters(),lr=0.001)

    for epoch in range(epochs):
        optimizer.zero_grad()
        ouput = model(X)
        loss = criteria(ouput,Y.float())
       
        loss.backward()
        optimizer.step()

        # Saving data points
        accuracy = (sum(abs(ouput - (Y.float())) < 0.5 ))/len(ouput)*100
        print(f" accuracy {accuracy}")
        plotAllModels(modelsConfigurations=[modelConfigurations[0]], dataframe=df)
        
      
        sleep(0.8)
        if epoch != epochs-1:
            display.clear_output(wait=True)
        accuracies.append(accuracy)
        losses.append(loss.item())
        print(f" epoch {epoch} loss : {loss.item()}")
        print(f" epoch {epoch} accuracy : {accuracy}")

print("OutSide the Loop")


# plt.figure(figsize=(400,400))
# plt.subplot(1,2,1)
# plt.title('Accuracy vs  epoch')
# plt.plot(accuracies)
# plt.subplot(1,2,2)
# plt.title('Loss vs  epoch')
# plt.plot(losses)

# plt.show()




 epoch 2 loss : 0.6978679895401001
 epoch 2 accuracy : tensor([51.])
 accuracy tensor([51.])
torch.Size([10000, 1]) torch.Size([10000, 1]) torch.Size([10000, 2])
 rows 1 cols 3
1 1


KeyboardInterrupt: 